# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [6]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [7]:
from sklearn.model_selection import train_test_split

# Create the dataframe split
X = adult_dt.drop(columns=['income'])  # All columns except 'income'
Y = adult_dt['income']                 # Only 'income' column

# Next Splitting the data into training and testing sets 70-30% with random seed of 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Show summary of the datasets
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((22792, 14), (9769, 14), (22792,), (9769,))

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The random state of a splitting function is a parameter that controls the randomness of the data split when dividing a dataset into training and testing sets (or other subsets). In Python's scikit-learn, the train_test_split() function includes a random_state parameter, which helps manage the random shuffling of data before splitting. It is essential for reproducibility, especially when testing or sharing code with others, as it allows consistent comparison of results across runs.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Separate numerical and categorical columns from the dataset
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Define the preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),  # KNN imputation
    ('scaler', RobustScaler())                                   # Scaling robust to outliers
])

# Define the preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),        # Imputation with mode
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  # One-hot encoding with binary drop
])

# Creating the column transformer to apply the transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),       # Applying numerical transformation
        ('cat', categorical_transformer, categorical_features)    # Applying categorical transformation
    ]
)

# Displaying the preprocessor setup
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [9]:
from sklearn.ensemble import RandomForestClassifier

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),         # Adding the preprocessing step
    ('classifier', RandomForestClassifier()) # Adding the classifier step
])

# Show the structure of the model pipeline
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [10]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation with 5 folds
cv_results = cross_validate(model_pipeline, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)

# Displaying the cross-validation results
cv_results

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


{'fit_time': array([7.05010986, 7.23224902, 6.20168877, 6.48217797, 6.80068111]),
 'score_time': array([0.33404279, 0.229074  , 0.21253228, 0.21854711, 0.21395802]),
 'test_neg_log_loss': array([-0.35641957, -0.38753577, -0.37522517, -0.34902113, -0.40108073]),
 'train_neg_log_loss': array([-0.081475  , -0.08155314, -0.08132542, -0.08216006, -0.08122152]),
 'test_roc_auc': array([0.9049376 , 0.90194691, 0.9018038 , 0.90789765, 0.90229193]),
 'train_roc_auc': array([1.        , 1.        , 0.99999998, 1.        , 1.        ]),
 'test_accuracy': array([0.85150252, 0.8466769 , 0.85103115, 0.86002633, 0.85805178]),
 'train_accuracy': array([0.99994515, 1.        , 0.99994516, 1.        , 1.        ]),
 'test_balanced_accuracy': array([0.77721446, 0.76726871, 0.77152807, 0.78361868, 0.77923859]),
 'train_balanced_accuracy': array([0.9998869 , 1.        , 0.99988693, 1.        , 1.        ])}

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [11]:
# Convert the cross-validation results into a DataFrame for readability
cv_results_df = pd.DataFrame(cv_results)

# Sort it by negative log loss of the test set as our most important indicator ('test_neg_log_loss')
cv_results_df_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=True)

# Display the sorted DataFrame
print(cv_results_df_sorted)

   fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4  6.800681    0.213958          -0.401081           -0.081222      0.902292   
1  7.232249    0.229074          -0.387536           -0.081553      0.901947   
2  6.201689    0.212532          -0.375225           -0.081325      0.901804   
0  7.050110    0.334043          -0.356420           -0.081475      0.904938   
3  6.482178    0.218547          -0.349021           -0.082160      0.907898   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4            1.0       0.858052        1.000000                0.779239   
1            1.0       0.846677        1.000000                0.767269   
2            1.0       0.851031        0.999945                0.771528   
0            1.0       0.851503        0.999945                0.777214   
3            1.0       0.860026        1.000000                0.783619   

   train_balanced_accuracy  
4                 1.000000  
1         

Calculate the mean of each metric. 

In [12]:
# Calculate the mean of each metric across the cross-validation folds
cv_means = cv_results_df.mean()

# Show the means of each metric
cv_means

fit_time                   6.753381
score_time                 0.241631
test_neg_log_loss         -0.373856
train_neg_log_loss        -0.081547
test_roc_auc               0.903776
train_roc_auc              1.000000
test_accuracy              0.853458
train_accuracy             0.999978
test_balanced_accuracy     0.775774
train_balanced_accuracy    0.999955
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [13]:
# Generate predictions and prediction probabilities
model_pipeline.fit(X_train, Y_train)  # Fit the pipeline on training data
Y_pred = model_pipeline.predict(X_test)  # Predictions for accuracy and balanced accuracy
Y_pred_proba = model_pipeline.predict_proba(X_test)  # Prediction probabilities for log loss and ROC AUC

# Calculate performance metrics on test data
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),                   # Negative log loss
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),              # ROC AUC with last column only
    'accuracy': accuracy_score(Y_test, Y_pred),                        # Accuracy
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)       # Balanced Accuracy
}

# Show the test metrics as a dictionary
test_metrics

{'neg_log_loss': -0.38435022679044367,
 'roc_auc': 0.9001120233356348,
 'accuracy': 0.8560753403623708,
 'balanced_accuracy': 0.7775548711456292}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable immediately upon loading ensures data consistency, saves time by preparing the dataset in a model-ready format, and improves code readability by making transformations clear from the start. This allows for a more efficient workflow

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.